<a href="https://colab.research.google.com/github/esmanureral/ArtFrame/blob/master/Keras_Custom_Layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Keras ile üç türlü model oluşturabiliriz.**

1) Sequential API
esnek değil kompleks model kurmak zordur.

2) Functional API
Çok girdi ve çok çıktı modeller kurmak için.Basit ve esnek

İkisi de deklaratiftir.Yani kullanmak istediğiniz katmanları ve bu katmanları nasıl bağlanacağını deklara ederek mimariyi oluştururuz.
--kolayca kaydedebilir,kopyalayabilir,paylaşabiliriz.
Dinamik modellerde pek uygun değildir.

3)Subclassing API
Daha çok akademiik araştırmalar için daha dinamiktir.Daha zor.

# **Subclassing** **API**

-önce rastgele başlangıç ağırlıkları üretildi

-Sonra bu değerler x girdisiyle matris çarpımı yapıldı ve çıkan sonucu bir bias eklendi.



In [1]:
import tensorflow as tf
from tensorflow import keras

# Custom Layer with Subclassing API

In [2]:
class Linear(keras.layers.Layer):#Linear sınıfı Keras Layers dan miras alıyor.
  def __init__(self, units=32, input_dim = 32):
    #init sınıftan örnek aldığımızda çalışacak.Bu metodun ilk parametresi selftir.Metoda nöron sayısı ve input boyutu verilir.
    super(Linear,self).__init__()

    #başlangıç ağırlıklarını oluşturalım:
    w_init= tf.random_normal_initializer()
    self.w=tf.Variable(
        initial_value=w_init(shape=(input_dim,units), dtype="float32"),
        trainable=True #ağırlıkların eğitilebilir olması için
    )
    #bias oluşturalım
    b_init = tf.zeros_initializer()
    self.b = tf.Variable(
        initial_value=b_init(shape=(units,), dtype="float32"),
        trainable=True
    )
  def call(self,inputs):
    #katmanımız linear oldugundan girdiler ve ağırlıklar matris çarpımıyla çarpılır.
    return tf.matmul(inputs, self.w) + self.b

In [3]:
#girdi verisi oluşturalım
x=tf.ones((2,2))
x

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 1.],
       [1., 1.]], dtype=float32)>

In [4]:
#bu linear sınıfından bir örnek alalım
linear_layer=Linear(4,2)
#veriyi bu katmandan geçirelim
y=linear_layer(x)
y

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[0.08621818, 0.0580382 , 0.09978607, 0.03098121],
       [0.08621818, 0.0580382 , 0.09978607, 0.03098121]], dtype=float32)>

In [5]:
linear_layer.weights


[]

# Custom Layer using add_weight
--katmana ağırlık ekleyelim.

In [6]:
class Linear(keras.layers.Layer):
  def __init__(self, units=32, input_dim = 32):
    super(Linear,self).__init__()
    self.w=self.add_weight(shape=(input_dim,units),
                           initializer="random_normal",
                           trainable=True)
    self.b=self.add_weight(shape=(units,),
                           initializer="zeros",
                           trainable=True)

  def call(self,inputs):
    return tf.matmul(inputs, self.w) + self.b

In [7]:
x = tf.ones((2,2))
linear_layer = Linear(4,2)
y = linear_layer(x)

In [8]:
y

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[0.0848397 , 0.02597267, 0.02139331, 0.03164777],
       [0.0848397 , 0.02597267, 0.02139331, 0.03164777]], dtype=float32)>

# Custom Layer without the shape of the inputs

birçok durumda girdilerimizin boyutunu önceden bilemeyebiliriz.

In [15]:
class Linear(keras.layers.Layer):
  def __init__(self,units=32):
    super(Linear,self).__init__()
    self.units=units
  def build(self,input_shape):
    self.w=self.add_weight(shape=(input_shape[-1],self.units),
                           initializer="random_normal",
                           trainable=True)
    self.b=self.add_weight(shape=(self.units,),
                           initializer="random_normal",
                           trainable=True)
  def call(self,inputs):
    return tf.matmul(inputs,self.w)+self.b

In [16]:
#katman örneği alalım
linear_layer=Linear(32) #nöron boyutu
y=linear_layer(x)
y

<tf.Tensor: shape=(2, 32), dtype=float32, numpy=
array([[ 0.03480168,  0.11780477, -0.07146603,  0.09356807,  0.00287807,
         0.016788  , -0.02560977, -0.050952  ,  0.00416769,  0.09172215,
        -0.02337775,  0.02248864, -0.02516513,  0.07314226,  0.02300491,
        -0.09440732,  0.15255876, -0.00376333, -0.01522268,  0.06076122,
         0.01635852, -0.06077199,  0.05912334,  0.03288104,  0.12464747,
         0.08507667,  0.05974192,  0.09405755,  0.16892117, -0.0013404 ,
         0.06889161,  0.05309429],
       [ 0.03480168,  0.11780477, -0.07146603,  0.09356807,  0.00287807,
         0.016788  , -0.02560977, -0.050952  ,  0.00416769,  0.09172215,
        -0.02337775,  0.02248864, -0.02516513,  0.07314226,  0.02300491,
        -0.09440732,  0.15255876, -0.00376333, -0.01522268,  0.06076122,
         0.01635852, -0.06077199,  0.05912334,  0.03288104,  0.12464747,
         0.08507667,  0.05974192,  0.09405755,  0.16892117, -0.0013404 ,
         0.06889161,  0.05309429]], dtyp

# Layers are recursively composable

Bazen ard arda gelen katmanları birleştirmek isteyebiliriz.

MLP dir.


In [17]:
class MLPBlock(keras.layers.Layer):
  def __init__(self):
    super(MLPBlock, self).__init__()
    self.linear_1 = Linear(32)
    self.linear_2 = Linear(32)
    self.linear_3 = Linear(1)
  def call(self, inputs):
    x = self.linear_1(inputs)
    x = tf.nn.relu(x)
    x = self.linear_2(x)
    x = tf.nn.relu(x)
    return self.linear_3(x)

In [18]:
mlp = MLPBlock()
y = mlp(tf.ones(shape=(3,64)))

In [19]:
mlp.weights

[<Variable path=mlp_block/linear_6/variable_4, shape=(64, 32), dtype=float32, value=[[ 0.04053406 -0.05016517  0.01516337 ...  0.01225795 -0.01203081
   -0.03253184]
  [-0.00952591  0.06968482  0.01553858 ... -0.11527377 -0.04697023
    0.06715441]
  [-0.04270023 -0.01283539  0.03078434 ... -0.01578767  0.02714171
   -0.01831546]
  ...
  [-0.02653     0.05365742 -0.0215816  ...  0.10659082 -0.00320693
   -0.14810663]
  [ 0.0273397   0.00937178  0.01770925 ... -0.01762283 -0.02773935
    0.04364376]
  [ 0.04169704 -0.01985934  0.06907679 ...  0.03022309  0.04251691
   -0.03191711]]>,
 <Variable path=mlp_block/linear_6/variable_5, shape=(32,), dtype=float32, value=[ 0.05304926  0.00774437 -0.03757168 -0.02044045  0.03601442 -0.03246492
  -0.02451975  0.02543026 -0.0441302   0.05765259 -0.08321667  0.03250953
   0.06687843 -0.00884438  0.01845261  0.03852453 -0.02811182  0.03455951
  -0.03736652  0.04717435 -0.05776708  0.04603823  0.07512236 -0.00754415
  -0.05295792  0.02957914 -0.10435